In [23]:
# 분석에 필요한 데이터 4개 불러오기
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

seoul_202112 = pd.read_csv('/Users/parkchangyu/Desktop/수업 자료/2023년 2학기/통데분/통데분 과제 /팀플/seoul_202112 .csv')
seoul_busan_201903 = pd.read_csv('/Users/parkchangyu/Desktop/수업 자료/2023년 2학기/통데분/통데분 과제 /팀플/seoul, busan_201903.csv')
seoul_covid19 = pd.read_excel('/Users/parkchangyu/Desktop/수업 자료/2023년 2학기/통데분/통데분 과제 /팀플/서울시구별코로나_확진자_사망자.xlsx')
obesity_20192021 = pd.read_csv('/Users/parkchangyu/Desktop/수업 자료/2023년 2학기/통데분/통데분 과제 /팀플/obesity_rate_2019_2021.csv', encoding='euc-kr')

#seoul_202112
#seoul_busan_201903
#obesity_20192021
#seoul_covid19

In [42]:
#업종별점포수변화
# seoul_busan_201903 전처리
select_col = ['상호명', '상권업종대분류코드', '상권업종중분류명', '시군구명', '상가업소번호']
seoul_busan_201903 = seoul_busan_201903[select_col]
seoul_busan_201903['년도'] = 2019
seoul_busan_201903 = seoul_busan_201903[seoul_busan_201903['상권업종대분류코드'] == 'Q']
seoul_busan_201903_subset = seoul_busan_201903[['상권업종중분류명', '년도']]


# seoul_202112 전처리
select_col = ['상호명', '상권업종대분류코드', '상권업종중분류명', '시군구명', '상가업소번호']
seoul_202112 = seoul_202112[select_col]
seoul_202112['년도'] = 2021
seoul_202112 = seoul_202112[seoul_202112['상권업종대분류코드'] == 'Q']
seoul_202112_subset = seoul_202112[['상권업종중분류명', '년도']]

df_combined = pd.concat([seoul_busan_201903_subset, seoul_202112_subset])

grouped_data = df_combined.groupby(['년도', '상권업종중분류명']).size().reset_index(name='Count')

# 오름차순 sort(Count 기준)
grouped_data = grouped_data.sort_values(by='Count', ascending=True)

grouped_data

,년도,상권업종중분류명,Count
21,2021,음식배달서비스,82
7,2019,음식배달서비스,147
17,2021,부페,419
14,2021,기타음식업,425
0,2019,기타음식업,499
3,2019,부페,631
16,2021,별식/퓨전요리,2409
24,2021,중식,3171
2,2019,별식/퓨전요리,3878
23,2021,제과제빵떡케익,4562


In [40]:
#2021년 지역구별 점포수
# seoul_202112 전처리
select_col = ['상호명', '상권업종대분류코드', '상권업종중분류명', '시군구명', '상가업소번호']
seoul_202112 = seoul_202112[select_col]
seoul_202112['년도'] = 2021
seoul_202112

seoul_202112_q = seoul_202112[seoul_202112['상권업종대분류코드'] == 'Q']

# Assuming seoul_202112_q is the filtered DataFrame with '상권업종대분류코드' equal to 'Q'

# '시군구명'으로 데이터를 그룹화하고 각 그룹 내에서 행의 수(점포의 수)를 계산
shop_data21 = seoul_202112_q.groupby('시군구명')['상호명'].count()

# 결과 출력: 각 구의 점포 수를 확인할 수 있음
shop_data21


# seoul_2019 전처리
# seoul_201903 전처리
seoul_201903 = seoul_busan_201903[select_col]
seoul_201903.loc[:, '년도'] = 2019
seoul_201903
seoul_201903_q = seoul_201903[seoul_201903['상권업종대분류코드'] == 'Q']


# '시군구명'으로 데이터를 그룹화하고 각 그룹 내에서 행의 수(점포의 수)를 계산
seoul_201903 = seoul_201903_q.groupby('시군구명')['상호명'].count()

drop_col = ['금정구', '기장군','남구','동구','동래구','부산진구','북구','사상구','사하구','서구','수영구','연제구','영도구','해운대구']
shop_data19 = seoul_201903.loc[~seoul_201903.index.isin(drop_col)]
# Merge grouped2019 and grouped2021 on '시군구명'
merged_df = pd.merge(shop_data19, shop_data21, left_index=True, right_index=True, suffixes=('_2019', '_2021'))

# 2021 값에서 2019 값의 차이 계산 및 새로운 컬럼에 저장
merged_df['차이'] = merged_df['상호명_2021'] - merged_df['상호명_2019']

# '상호명_2019' 컬럼명을 '19점포수', '상호명_2021' 컬럼명을 '21점포수'로 변경
shop_data = merged_df.rename(columns={'상호명_2019': '19점포수', '상호명_2021': '21점포수'})

shop_data

/var/folders/fd/p8gl394x5wjgxs6cdb2744br0000gn/T/ipykernel_29668/2109493407.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seoul_201903.loc[:, '년도'] = 2019


,19점포수,21점포수,차이
시군구명,,,
강남구,15381,10920,-4461
강동구,4994,4393,-601
강북구,3154,3126,-28
강서구,7827,5781,-2046
관악구,4499,4161,-338
광진구,6786,4169,-2617
구로구,4359,4029,-330
금천구,2918,2778,-140
노원구,4204,3695,-509


In [25]:
#연도별,지역구별 비만율 
# obesity_20192021 전처리
#obesity_20192021

# 0, 1, 2 행만 선택하여 새로운 DataFrame 생성
obesity_subset = obesity_20192021.loc[[0, 1, 2]]

# 결과 출력
obesity_subset

# 성별, 19~29세, 30~44세, 45~64세, 65세 이상 열을 제외한 열만 남기기
obesity_data = obesity_subset.drop(columns=['성별', '19~29세', '30~44세', '45~64세', '65세 이상'])


# 결과 출력
obesity_data

,년도,소계,종로구,중구,용산구,성동구,광진구,동대문구,중랑구,성북구,...,강서구,구로구,금천구,영등포구,동작구,관악구,서초구,강남구,송파구,강동구
0,2019,31.8,34.2,29.1,26.5,32.4,28.9,36.1,36.2,32.6,...,30.7,33.0,35.2,34.3,30.0,35.1,25.1,28.6,27.9,31.3
1,2020,28.9,28.7,26.2,25.5,29.6,25.1,31.3,31.9,28.4,...,30.2,29.6,30.5,28.3,29.9,27.1,24.7,25.6,27.0,31.8
2,2021,28.6,25.4,27.2,26.2,26.3,27.6,31.3,31.0,28.2,...,29.6,28.0,34.9,28.0,28.8,29.5,23.5,22.5,27.2,27.1


In [26]:
#코로나 확진자 데이터
# seoul_covid19 전처리
seoul_covid19.columns = ['날짜', '총합계 확진', '총합계 사망', '강남구 확진', '강남구 사망', '강동구 확진', '강동구 사망',
    '강북구 확진', '강북구 사망', '강서구 확진', '강서구 사망', '관악구 확진', '관악구 사망', '광진구 확진',
       '광진구 사망', '구로구 확진', '구로구 사망', '금천구 확진', '금천구 사망', '노원구 확진',
       '노원구 사망', '도봉구 확진', '도봉구 사망', '동대문구 확진', '동대문구 사망', '동작구 확진',
       '동작구 사망', '마포구 확진', '마포구 사망', '서대문구 확진', '서대문구 사망', '서초구 확진',
       '서초구 사망', '성동구 확진', '성동구 사망', '성북구 확진', '성북구 사망', '송파구 확진',
       '송파구 사망', '양천구 확진', '양천구 사망', '영등포구 확진', '영등포구 사망', '용산구 확진',
       '용산구 사망', '은평구 확진', '은평구 사망', '종로구 확진', '종로구 사망', '중구 확진',
       '중구 사망', '중랑구 확진', '중랑구 사망', '기타 확진', '기타 사망', '타시도 확진',
       '타시도 사망']
seoul_covid19 = seoul_covid19.drop(0)
drop_col = ['기타 확진', '기타 사망', '타시도 확진', '타시도 사망']
seoul_covid19 = seoul_covid19.drop(columns=drop_col)
seoul_covid19 = seoul_covid19.iloc[:-1]
seoul_covid19

,날짜,총합계 확진,총합계 사망,강남구 확진,강남구 사망,강동구 확진,강동구 사망,강북구 확진,강북구 사망,강서구 확진,...,용산구 확진,용산구 사망,은평구 확진,은평구 사망,종로구 확진,종로구 사망,중구 확진,중구 사망,중랑구 확진,중랑구 사망
1,2020-01-24 00:00:00,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-01-30 00:00:00,3,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,2020-01-31 00:00:00,3,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
4,2020-02-02 00:00:00,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2020-02-05 00:00:00,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,2023-08-27 00:00:00,1905,1,121,0,116,0,32,0,118,...,32,0,165,0,10,0,29,0,41,0
1281,2023-08-28 00:00:00,8485,1,452,0,459,0,223,0,503,...,177,0,377,0,99,0,104,0,355,0
1282,2023-08-29 00:00:00,7470,5,399,0,382,0,220,1,411,...,145,0,347,0,98,0,79,0,286,0
1283,2023-08-30 00:00:00,6423,3,308,0,305,0,156,0,375,...,105,1,348,0,69,0,82,0,256,0


In [36]:
#지역구별 코로나 확진자 데이터
# seoul_covid19 전처리
# '날짜' 열을 날짜 형식으로 변환
seoul_covid19['날짜'] = pd.to_datetime(seoul_covid19['날짜'])

# '월' 열을 추가하여 월 정보 추출
seoul_covid19['년'] = seoul_covid19['날짜'].dt.to_period('Y')

# 월별로 데이터 합치기
covid19live = seoul_covid19.groupby('년').agg({
    '총합계 확진': 'sum',
    '총합계 사망': 'sum',
    '강남구 확진': 'sum',
    '강동구 확진': 'sum',
    '강북구 확진': 'sum',
    '강서구 확진': 'sum',
    '관악구 확진': 'sum',
    '광진구 확진': 'sum',
    '구로구 확진': 'sum', 
    '금천구 확진': 'sum',
    '노원구 확진': 'sum',
    '도봉구 확진': 'sum',
    '동대문구 확진': 'sum',
    '동작구 확진': 'sum',
    '마포구 확진': 'sum',
    '서대문구 확진': 'sum',
    '서초구 확진': 'sum',
    '성동구 확진': 'sum',
    '성북구 확진': 'sum',
    '송파구 확진': 'sum',
    '양천구 확진': 'sum',
    '영등포구 확진': 'sum',
    '용산구 확진': 'sum',
    '은평구 확진': 'sum', 
    '종로구 확진': 'sum',
    '중구 확진': 'sum',
    '중랑구 확진': 'sum'
    
}).reset_index()

# 결과 출력
covid19live

,년,총합계 확진,총합계 사망,강남구 확진,강동구 확진,강북구 확진,강서구 확진,관악구 확진,광진구 확진,구로구 확진,...,성동구 확진,성북구 확진,송파구 확진,양천구 확진,영등포구 확진,용산구 확진,은평구 확진,종로구 확진,중구 확진,중랑구 확진
0,2020,19349,182,934,579,444,1339,1017,461,620,...,449,812,1118,724,692,419,786,403,280,802
1,2021,207215,1631,11589,8728,6288,9166,10296,6591,10078,...,5298,8732,12603,7310,9838,4890,8926,3583,3412,7729
2,2022,5445016,4252,277131,250516,146284,314400,269507,189129,216744,...,156325,232205,357007,239227,218468,111880,246645,71909,64545,199374
3,2023,1079643,610,52428,48022,28622,63725,55677,35061,41869,...,28976,42853,69720,44434,42845,21314,52158,14534,13163,38686


In [35]:
#지역구별 코로나 확진자 데이터
# seoul_covid19 전처리
# '날짜' 열을 날짜 형식으로 변환

# 월별로 데이터 합치기
covid19dead = seoul_covid19.groupby('년').agg({
    '총합계 확진': 'sum',
    '총합계 사망': 'sum',
    '강남구 사망': 'sum',
    '강동구 사망': 'sum',
    '강북구 사망': 'sum',
    '강서구 사망': 'sum',
    '관악구 사망': 'sum',
    '광진구 사망': 'sum',
    '구로구 사망': 'sum', 
    '금천구 사망': 'sum',
    '노원구 사망': 'sum',
    '도봉구 사망': 'sum',
    '동대문구 사망': 'sum',
    '동작구 사망': 'sum',
    '마포구 사망': 'sum',
    '서대문구 사망': 'sum',
    '서초구 사망': 'sum',
    '성동구 사망': 'sum',
    '성북구 사망': 'sum',
    '송파구 사망': 'sum',
    '양천구 사망': 'sum',
    '영등포구 사망': 'sum',
    '용산구 사망': 'sum',
    '은평구 사망': 'sum', 
    '종로구 사망': 'sum',
    '중구 사망': 'sum',
    '중랑구 사망': 'sum'
    
}).reset_index()

# 결과 출력
covid19dead

,년,총합계 확진,총합계 사망,강남구 사망,강동구 사망,강북구 사망,강서구 사망,관악구 사망,광진구 사망,구로구 사망,...,성동구 사망,성북구 사망,송파구 사망,양천구 사망,영등포구 사망,용산구 사망,은평구 사망,종로구 사망,중구 사망,중랑구 사망
0,2020,19349,182,5,2,1,8,8,6,3,...,7,15,8,2,10,1,7,4,3,5
1,2021,207215,1631,48,99,51,71,81,27,68,...,30,88,75,77,78,31,84,32,20,58
2,2022,5445016,4252,145,252,151,247,198,134,219,...,106,207,212,154,177,74,169,51,45,153
3,2023,1079643,610,22,17,38,28,29,29,19,...,17,38,28,21,27,10,17,5,4,25
